In [ ]:
from __future__ import annotations        # make Python behave
import numpy as np                        # standard array library
import time                               # timers
import sys                                # add DSL library to the Python path
sys.path.append(sys.path[0]+"/..")
from SYS_ATL import proc                  # import the SYS-ATL DSL

In [ ]:
# I'm going to define a 1-d version of a standard convolutional layer, like in CuDNN

# K - # of output channels
# C - # of input channels
# W - length of the input signal/tensor
# R - width of the filter kernel

@proc
def conv1d(K : size, C : size, W : size, R : size,
           w : R[K,C,R],  # filter kernel weights
           x : R[C,W],    # input signal
           res : R[K,W],  # output signal
):
    # zero out the result memory
    for k_init in par(0,K):
        for i_init in par(0,W):
            res[k_init, i_init] = 0.0
    
    # do the convolution
    for k in par(0,K):
        for c in par(0,C):
            for i in par(0,W):
                for r in par(0,R):
                    if 0 <= i-r:
                        res[k,i] += w[k,c,r] * x[c,i-r]

conv1d

In [ ]:
# One important early method of mapping Neural-Network Convolutions
# onto high-performance Dense-Matrix-Multipliation primitives
# was called `im2col` based on a related Matlab operation

# Usually this transformation is explained with complex figures such as the following:

<img src="im2col_explanation.png">
image from: https://leonardoaraujosantos.gitbook.io/artificial-
inteligence/machine_learning/deep_learning/convolution_layer/making_faster

In [ ]:
# Let me show you a more straight-forwardly algebraic way
# of understanding why im2col is an effective strategy
# for computing convolutions

# first, let's look at the inner statement of our convolution

```python
res[k, i] += w[k, c, r] * x[c, i - r]
```

In [ ]:
# If instead of `x[c, i-r]` we had `x[c, r, i]`
# then treating `c,r` as a single variable,
# this would look like the inner loop of a standard
# dense-matrix-multiply

```python
res[k, i] += w[k, c,r] * x[c,r, i]

# i.e. as matrices

RES = W * X
```

In [ ]:
# Let's start applying scheduling
im2col_conv = ( conv1d.rename('im2col_conv')
                      .reorder('i','r')
                      .bind_expr('y','x[c, i-r]')
              )
im2col_conv

In [ ]:
# notice the `y : R` line which means
# "create a new buffer with type Real-Number"
# i.e. we are allocating a local scalar variable

In [ ]:
# next, we can start to lift that allocation
# up and out of the loop
im2col_conv.lift_alloc('y:R', 5)

In [ ]:
im2col_conv = im2col_conv.lift_alloc('y:R', 5)

In [ ]:
# Then, we can fission the loop correspondingly,
# separating what is now a data-marshalling statement from
# the actual compute statement in two subsequent
# loop nests via fissioning
im2col_conv.fission_after('y[c,r,i] = _',3)

In [ ]:
im2col_conv = im2col_conv.fission_after('y[c,r,i] = _',5)

In [ ]:
im2col_conv

In [ ]:
# Now, in order to expose these two parts of the computation as
# re-usable sub-procedures, we want a way to factor them out.
im2col_conv, im2col = im2col_conv.factor_out_stmt('im2col', 'for c in _: _')
im2col_conv, matmul = im2col_conv.factor_out_stmt('matmul', 'for k in _: _')
im2col_conv

In [ ]:
im2col

In [ ]:
matmul

In [ ]:
# Given this factoring, we can then proceed
# to schedule these sub-procedures themselves.
tiled_matmul =      (matmul.rename('tiled_matmul')
                     # split the loops we want to tile together
                     .reorder('r','i')
                     .split('k',8,['khi','klo'], cut_tail=True)
                     .reorder('klo[1]','c').reorder('klo[1]','i')
                     .split('c[1]',8,['chi','clo'], cut_tail=True)
                     .reorder('clo[1]','i').reorder('clo[1]','klo')
                     .split('i[1]', 8, ['ihi','ilo'], cut_tail=True)
                     .reorder('ilo[1]','klo').reorder('ilo[1]','clo'))
tiled_matmul

In [ ]:
# However, note that the convolution still calls the original matmul
im2col_conv

In [ ]:
# We can invoke another scheduling directive
# to change which version of the matmul gets scheduled
im2col_conv.call_eqv(tiled_matmul, 'matmul(_,_,_,_,_,_,_)')

In [ ]:
# Note!
# Crucially this is only allowed because we know that
#    matmul == tiled_matmul
# by construction.